In [ ]:
import os
import json
import re
from pathlib import Path

import feedparser
import pandas as pd
from datetime import datetime, timedelta
from newspaper import Article
from perplexity import Perplexity
from dotenv import load_dotenv
from vnstock import Listing, Quote, Trading, Company, Vnstock

In [3]:
load_dotenv("/Users/rzy/Desktop/ProjectWithTien/RAG/stock-me-2/server/.env")

True

In [4]:
# --- CONFIG ---
# register_user(api_key=os.getenv("VN_STOCK_API_KEY"))

# Get your Key: https://www.perplexity.ai/settings/api
MODEL_ID = "sonar"  # Perplexity model (or "sonar-pro" for pro version)
WATCHLIST = ["FPT"]

# Initialize Perplexity client (automatically uses PERPLEXITY_API_KEY from env)
client = Perplexity(api_key=os.getenv("PERPLEXITY_API_KEY"))

In [5]:
symbol = "FPT"

In [9]:
signals = {
    "dividend_alert": None,  # Will store text like "10% Cash (1,000đ)"
    "ex_date": None,         # The deadline to buy
    "pe_ratio": None,        # Valuation (Cheap vs Expensive)
    "events": []             # Recent news/events
}

# 1. Setup Client (VCI source is usually best for fundamentals)
company = Company(symbol=symbol, source='VCI')
stock = Vnstock().stock(symbol=symbol, source="VCI")
# 1) Giá lịch sử (để lấy close)
price = stock.quote.history(start="2022-01-01", end="2025-12-31", interval="1D")
price["time"] = pd.to_datetime(price["time"], errors="coerce")
price = price.sort_values("time")

# --- SIGNAL B: EVENTS (The "News" Check) ---
# This is the function you found in the docs
events_df = company.events(page_size=40)

stock_div = events_df[
    events_df["event_title"].str.contains("cổ tức", case=False, na=False)
    & events_df["event_title"].str.contains("cổ phiếu", case=False, na=False)
].copy()
stock_div["exright_date"] = pd.to_datetime(stock_div["exright_date"], errors="coerce")


# 3) Lấy close của ngày gần nhất <= exright_date (xấp xỉ “giá trước GDKHQ”)
price2 = price.rename(columns={"time": "date"})
price2 = price2[["date", "close"]].dropna()

stock_div = stock_div.sort_values("exright_date")
price2 = price2.sort_values("date")

merged = pd.merge_asof(
    stock_div,
    price2,
    left_on="exright_date",
    right_on="date",
    direction="backward"
)

print(merged.columns)

# 4) Tính yield
merged["dividend_yield"] = merged["value"] / merged["close"]

# Only keep what matters for dividend cash or stock display
keep_cols = ['exright_date', "event_title", 'en__event_list_name', 'value', 'ratio', 'close', 'dividend_yield']
merged = merged[keep_cols]

print(merged.tail(10))

Index(['id', 'event_title', 'en__event_title', 'public_date', 'issue_date',
       'source_url', 'event_list_code', 'ratio', 'value', 'record_date',
       'exright_date', 'event_list_name', 'en__event_list_name', 'date',
       'close'],
      dtype='object')
   exright_date                                        event_title  \
7    2022-06-13      FPT - Phát hành cổ phiếu trả cổ tức tỷ lệ 20%   
8    2022-06-13  FPT - Trả cổ tức Đợt 2 năm 2021 bằng tiền 1000...   
9    2022-08-24  FPT - Trả cổ tức Đợt 1 năm 2022 bằng tiền 1000...   
10   2023-07-05  FPT - Trả cổ tức Đợt 2 năm 2022 bằng tiền 1000...   
11   2023-07-05      FPT - Phát hành cổ phiếu trả cổ tức tỷ lệ 15%   
12   2023-08-24  FPT - Trả cổ tức Đợt 1 năm 2023 bằng tiền 1000...   
13   2024-06-12  FPT - Trả cổ tức Đợt 2 năm 2023 bằng tiền 1000...   
14   2024-12-02  FPT - Trả cổ tức Đợt 1 năm 2024 bằng tiền 1000...   
15   2025-06-12  FPT - Trả cổ tức Đợt 2 năm 2024 bằng tiền 1000...   
16   2025-12-01  FPT - Trả cổ tức Đợt 1

In [16]:
def get_stock_dividends(symbol):
    """
    Fetches dividend events (cash and stock), matches them with historical price
    to calculating approximate yield at the time of the event.
    """
    try:
        symbol = symbol.upper()
        
        # 1. Fetch Company Events using vnstock
        company = Company(symbol=symbol, source='VCI')
        try:
            events_df = company.events(page_size=50) # Fetch enough recent events
        except Exception:
            # Fallback if VCI fails or empty
            return _err(f"No event data available for {symbol}")

        if events_df is None or events_df.empty:
             return _ok({"symbol": symbol, "data": []})

        # 2. Filter for Dividends (Cash or Stock)
        # Keywords: "cổ tức" (dividend), "trả" (pay), "thưởng" (bonus/reward)
        dividend_mask = (
            events_df["event_title"].str.contains("cổ tức", case=False, na=False) |
            events_df["event_title"].str.contains("thưởng", case=False, na=False) |
             events_df["event_title"].str.contains("trả", case=False, na=False)
        )
        
        div_df = events_df[dividend_mask].copy()
        
        if div_df.empty:
             return _ok({"symbol": symbol, "data": []})

        # Ensure dates are datetime objects
        div_df["exright_date"] = pd.to_datetime(div_df["exright_date"], errors="coerce")
        div_df = div_df.dropna(subset=['exright_date']).sort_values("exright_date")

        # 3. Fetch History to calculate Yield (Yield = Dividend Value / Price at Ex-Date)
        # We need a range covering the oldest event to today
        min_date = div_df["exright_date"].min()
        start_str = (min_date - timedelta(days=5)).strftime("%Y-%m-%d")
        end_str = datetime.now().strftime("%Y-%m-%d")

        quote = Quote(source="VCI", symbol=symbol)
        price_df = quote.history(start=start_str, end=end_str, interval="1D")

        results = []
        
        if not price_df.empty:
            price_df["time"] = pd.to_datetime(price_df["time"], errors="coerce")
            price_df = price_df.sort_values("time")
            
            # Prepare for merge_asof
            price_sub = price_df[["time", "close"]].rename(columns={"time": "date"}).dropna()
            
            # Merge events with price (find last price <= exright_date)
            merged = pd.merge_asof(
                div_df,
                price_sub,
                left_on="exright_date",
                right_on="date",
                direction="backward" 
            )

            # Process individual rows
            for _, row in merged.iterrows():
                # Extract value
                val = row.get('value')
                ratio = row.get('ratio')
                
                # Try convert value to float, handle '0' or None
                try:
                    val_float = float(val) if val else 0.0
                except (ValueError, TypeError):
                    val_float = 0.0
                
                # Calculate Yield if it's a cash dividend (value > 0)
                div_yield = 0.0
                close_price = row.get('close')
                
                if val_float > 0 and close_price and close_price > 0:
                    div_yield = (val_float / close_price) * 100

                # Determine Type
                event_type = "STOCK"
                if "tiền" in str(row.get("event_title", "")).lower() or val_float > 0:
                    event_type = "CASH"

                results.append({
                    "date": row["exright_date"].strftime("%Y-%m-%d"),
                    "title": row.get("event_title"),
                    "type": event_type,
                    "value": val_float,
                    "ratio": ratio,
                    "price_at_ex": close_price,
                    "yield_percent": round(div_yield, 2) if event_type == "CASH" else None
                })
        else:
             # If price history fails, return events without yield info
             for _, row in div_df.iterrows():
                results.append({
                    "date": row["exright_date"].strftime("%Y-%m-%d"),
                    "title": row.get("event_title"),
                    "type": "UNKNOWN",
                    "value": row.get('value'),
                    "ratio": row.get('ratio'),
                    "price_at_ex": None,
                    "yield_percent": None
                })

        # Sort by date descending
        results.sort(key=lambda x: x["date"], reverse=True)
        
        return results

    except Exception as e:
        print(f"Error fetching dividends for {symbol}: {e}")

In [17]:
get_stock_dividends("FPT")

[{'date': '2025-12-01',
  'title': 'FPT - Trả cổ tức Đợt 1 năm 2025 bằng tiền 1000 đồng/cổ phiếu',
  'type': 'CASH',
  'value': 1000.0,
  'ratio': 0.1,
  'price_at_ex': 96.6,
  'yield_percent': 1035.2},
 {'date': '2025-07-21',
  'title': 'FPT - Phát hành cổ phiếu thưởng tỷ lệ 15%',
  'type': 'STOCK',
  'value': 0.0,
  'ratio': 0.15,
  'price_at_ex': 109.16,
  'yield_percent': None},
 {'date': '2025-06-12',
  'title': 'FPT - Trả cổ tức Đợt 2 năm 2024 bằng tiền 1000 đồng/cổ phiếu',
  'type': 'CASH',
  'value': 1000.0,
  'ratio': 0.1,
  'price_at_ex': 100.7,
  'yield_percent': 993.05},
 {'date': '2024-12-02',
  'title': 'FPT - Trả cổ tức Đợt 1 năm 2024 bằng tiền 1000 đồng/cổ phiếu',
  'type': 'CASH',
  'value': 1000.0,
  'ratio': 0.1,
  'price_at_ex': 121.34,
  'yield_percent': 824.13},
 {'date': '2024-06-12',
  'title': 'FPT - Phát hành cổ phiếu thưởng tỷ lệ 15%',
  'type': 'STOCK',
  'value': 0.0,
  'ratio': 0.15,
  'price_at_ex': 111.86,
  'yield_percent': None},
 {'date': '2024-06-12'